In [1]:
import pandas as pd

In [2]:
with open('geo_data/cols.txt') as fp:
    headers = [line.split(':')[0].strip() for line in fp]
cols = ['name', 'country code', 'timezone', 'population']
df = pd.read_csv(filepath_or_buffer='geo_data/allCountries.txt',
                 sep='\t', names=headers, usecols=cols)


In [3]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12368093 entries, 0 to 12368092
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   name          object
 1   country code  object
 2   population    int64 
 3   timezone      object
dtypes: int64(1), object(3)
memory usage: 2.5 GB


In [4]:
pop_lim = 1000
best_matches = df.sort_values(by='population', ascending=False).drop_duplicates(subset='name') \
.rename(columns={'country code':'country_code'}) \
.query('population > @pop_lim and not timezone.isna() and not country_code.isna()') \
.query('name.str.fullmatch(r"[ -~]+")') \
.reset_index(drop=True)

In [16]:
best_matches

,name,country_code,population,timezone
0,Commonwealth of Nations,GB,2400000000,Europe/London
1,Indian Subcontinent,IN,1700000000,Asia/Kolkata
2,Republic of India,IN,1352617328,Asia/Kolkata
3,Islamic Republic of Pakistan,PK,212215030,Asia/Karachi
4,Uttar Pradesh,IN,199812341,Asia/Kolkata
...,...,...,...,...
123633,Lodno,SK,1001,Europe/Bratislava
123634,Yashresh,IL,1001,Asia/Jerusalem
123635,Sigiriya,LK,1001,Asia/Colombo
123636,Mirepoix-sur-Tarn,FR,1001,Europe/Paris


In [5]:
best_matches.to_json(path_or_buf='geo_data/locations.json')

In [6]:
country = pd.read_csv('geo_data/country_info.txt', sep='\t', usecols=['ISO', 'Country', 'Continent'])

In [7]:
t = pd.read_json('geo_data/locations.json')

In [8]:
locations_with_country = t.merge(right=country, left_on='country_code', right_on='ISO')

In [9]:
continent = pd.read_csv('geo_data/continent_codes.csv')

In [10]:
locations_with_continents = locations_with_country.merge(right=continent, left_on='Continent', right_on='Code') \
.drop(columns=['country_code', 'ISO', 'Continent', 'Code']) \
.rename(columns={'Name':'continent', 'Country': 'country'})

In [11]:
from datetime import datetime
from zoneinfo import ZoneInfo
tz_map = pd.DataFrame(dict(timezone=list(set(locations_with_continents.timezone))))
tz_map['tz_name'] = tz_map.timezone.apply(lambda tz: datetime.now(tz=ZoneInfo(tz)).strftime('%Z')) # zone name
tz_map['utc_offset'] = tz_map.timezone.apply(lambda tz: datetime.now(tz=ZoneInfo(tz)).strftime('%z')) # zone offset

In [12]:
tz_map.to_json('geo_data/tz_map.json')
locations_with_continents.to_json('geo_data/locs_with_continent.json')

In [13]:
x = pd.read_json('geo_data/tz_map.json', dtype='string')

In [15]:
x

,timezone,tz_name,utc_offset
0,Europe/Vienna,CET,+0100
1,America/Winnipeg,CST,-0600
2,Pacific/Apia,+13,+1300
3,America/Monterrey,CST,-0600
4,America/Indiana/Winamac,EST,-0500
...,...,...,...
377,Pacific/Kosrae,+11,+1100
378,Asia/Novosibirsk,+07,+0700
379,Europe/Chisinau,EET,+0200
380,America/Sitka,AKST,-0900
